In [1]:
from fjss import FJSS2, FJSS3
import numpy as np
import time

from itertools import product
from collections import OrderedDict

from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

from utils import get_m_value, parse_data
import pandas as pd

In [2]:
infinity = 1.0e6

In [3]:
def single_run(n_opt_selected=10, method="CP"):
    # load the data
    (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    ) = load_data()

    n_opt = n_opt_selected

    para_p_horizon = np.copy(para_p[:n_opt_selected, :])
    para_p_horizon[para_p_horizon == np.inf] = 0

    # para_h = np.empty((n_opt, n_mach), dtype=object)
    para_h_horizon = np.copy(para_h[:n_opt_selected, :])
    para_h_horizon[para_h_horizon == np.inf] = 0

    para_lmax_horizon = np.copy(para_lmax[:n_opt_selected, :n_opt_selected])
    para_lmax_horizon[para_lmax_horizon == np.inf] = 0

    horizon = (
        np.sum(para_p_horizon, axis=1)
        + np.sum(para_h_horizon, axis=1)
        + np.sum(para_lmax_horizon, axis=1)
    )
    horizon = int(np.sum(horizon)) + 1

    para_lmin[para_lmin == -np.inf] = -infinity
    para_lmin[para_lmin == np.inf] = infinity
    para_lmax[para_lmax == np.inf] = infinity
    para_lmax[para_lmax == -np.inf] = -infinity

    para_p[para_p == np.inf] = infinity
    para_p[para_p == -np.inf] = -infinity
    para_w[para_w == np.inf] = infinity
    para_w[para_w == -np.inf] = -infinity

    para_a[para_a == np.inf] = infinity
    para_a[para_a == -np.inf] = -infinity

    # =====================================================
    # convert all the numpy arrays with integers data type
    para_lmin = para_lmin.astype(int)
    para_lmax = para_lmax.astype(int)
    para_p = para_p.astype(int)
    para_w = para_w.astype(int)
    para_a = para_a.astype(int)
    para_delta = para_delta.astype(int)
    para_mach_capacity = para_mach_capacity.astype(int)

    para_lmin = para_lmin[:n_opt_selected, :n_opt_selected]
    para_lmax = para_lmax[:n_opt_selected, :n_opt_selected]

    para_p = para_p[:n_opt_selected, :]
    para_h = para_h[:n_opt_selected, :]
    para_w = para_w[:n_opt_selected, :]

    para_a = para_a[:n_opt_selected, :n_opt_selected, :]

    # machines
    machines = [str(i) for i in range(6)]
    # operations
    operations = [str(i) for i in range(n_opt_selected)]

    operations = operations[:n_opt_selected]
    machines = machines[:n_mach]

    # =====================================================
    if method == "CP":
        # time the running time for the CP model
        start_time = time.time()
        fjss2 = FJSS2(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_cp=1.0e6,
            num_workers=4,
            verbose=True,
        )
        fjss2.build_model_ortools()
        fjss2.solve_ortools()
        # get the running time in seconds
        running_time = time.time() - start_time

        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": 0,
            "makespan": fjss2.var_c_max,
        }
    elif method == "MILP":
        para_a = np.einsum("mij->ijm", para_a)
        # time the running time for the MILP model
        start_time = time.time()
        fjss3 = FJSS3(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_milp=1.0e7,
            num_workers=4,
            verbose=True,
        )
        fjss3.solve_gurobi()
        running_time = time.time() - start_time
        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": fjss3.solver.NumConstraints(),
            "makespan": fjss3.var_c_max,
        }
    else:
        raise ValueError("Invalid method!")

    return new_row


def load_data():
    n_opt, n_mach, operation_data, machine_data = parse_data(
        "fjss_cp_data/gfjsp_10_5_1.txt"
    )
    operation_data["0"]["h"] = 0
    operation_data["1"]["h"] = 0
    operation_data["2"]["h"] = 0
    operation_data["3"]["h"] = 20

    # define the parameters

    # minimum lag between the starting time of operation i and the ending time of operation j
    para_lmin = np.full((n_opt, n_opt), dtype=object, fill_value=-np.inf)
    # maximum lag between the starting time of operation i and the ending time of operation j
    para_lmax = np.full((n_opt, n_opt), dtype=object, fill_value=np.inf)

    # processing time of operation i in machine m
    # para_p = np.full((n_opt, n_mach), dtype=object, fill_value=np.inf)
    para_p = np.full((n_mach, n_opt), dtype=object, fill_value=np.inf)

    # the shape of h in the original file is (n_machine) while the shape of para_h in the
    # paper is (n_opt, n_mach)
    # maximum holding time of operation i in machine m
    para_h = np.empty((n_opt, n_mach), dtype=object)
    # para_h = np.empty(n_mach, dtype=object)

    # mapping of operation i to machine m
    # 20 for the furnaces, 0 for Cutting, Pressing, and Forging
    # 0: Cutting; 1: Pressing; 2: Forging; 3: Furnace
    holding_time_dict = {
        "0": 0,
        "1": 0,
        "2": 0,
        "3": 20,
    }

    # weight of operation i in machine m
    # para_w = np.empty((n_opt, n_mach), dtype=object)
    para_w = np.full((n_mach, n_opt), dtype=object, fill_value=np.inf)

    # input/output delay time between two consecutive operations in mahcine m
    para_delta = np.empty((n_mach), dtype=object)

    # setup time of machine m when processing operation i before j
    # para_a = np.full((n_opt, n_opt, n_mach), dtype=object, fill_value=np.inf)
    para_a = np.full((n_mach, n_opt, n_opt), dtype=object, fill_value=np.inf)

    # capacity of machine
    para_mach_capacity = np.empty((n_mach), dtype=object)
    for m in range(n_mach):
        # capacity of machine is a set of constant numbers
        para_mach_capacity[m] = machine_data[str(m)]["c"]

        # input/output delay time between two consecutive operations in mahcine m
        # delta(m): loading and unloading time of machine m (=1 for all machines)
        para_delta[m] = 1

        # set up time of machine m when processing operation i before j
        # a(i,j,m): setup time of machine m when processing operation i before j (aijm = -inf if there
        # is no setups)
        for idx_setup, setup_data in enumerate(machine_data[str(m)]["setup_data"][0]):
            para_a[m, int(setup_data[0]), int(setup_data[1])] = setup_data[2]

        # maximum holding time of operation i in machine m
        para_h[:, m] = holding_time_dict[str(machine_data[str(m)]["t"])]

    # lag time
    for i in range(n_opt):
        for idx_lag, lag_data in enumerate(operation_data[str(i)]["lag"]):
            # minimum lag between the starting time of operation i and the ending time of operation j
            para_lmin[i, int(lag_data[0])] = lag_data[1]
            # maximum lag between the starting time of operation i and the ending time of operation j
            para_lmax[i, int(lag_data[0])] = lag_data[2]

        for idx_pw, pw_data in enumerate(operation_data[str(i)]["pw"]):
            # operation_data[str(1)]["pw"]
            # # the shape of para_p in the original file is the transpose of the shape of para_p
            # para_p[i, int(pw_data[0])] = pw_data[1]
            # # the shape of para_w in the original file is the transpose of the shape of para_w
            # para_w[i, int(pw_data[0])] = pw_data[2]

            # the shape of para_p in the original file is the transpose of the shape of para_p
            para_p[int(pw_data[0]), i] = pw_data[1]
            # the shape of para_w in the original file is the transpose of the shape of para_w
            para_w[int(pw_data[0]), i] = pw_data[2]

    # reformat the shape of para_p and para_w
    para_p = para_p.T
    para_w = para_w.T

    # # reshape the shape of para_a
    # para_a = np.einsum("mij->ijm", para_a)

    # the big M value
    big_m = get_m_value(
        para_p=para_p, para_h=para_h, para_lmin=para_lmin, para_a=para_a
    )

    return (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    )

In [4]:
n_opt_selected = 50
method = "MILP"

In [5]:
    (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    ) = load_data()

    n_opt = n_opt_selected

    para_p_horizon = np.copy(para_p[:n_opt_selected, :])
    para_p_horizon[para_p_horizon == np.inf] = 0

    # para_h = np.empty((n_opt, n_mach), dtype=object)
    para_h_horizon = np.copy(para_h[:n_opt_selected, :])
    para_h_horizon[para_h_horizon == np.inf] = 0

    para_lmax_horizon = np.copy(para_lmax[:n_opt_selected, :n_opt_selected])
    para_lmax_horizon[para_lmax_horizon == np.inf] = 0

    horizon = (
        np.sum(para_p_horizon, axis=1)
        + np.sum(para_h_horizon, axis=1)
        + np.sum(para_lmax_horizon, axis=1)
    )
    horizon = int(np.sum(horizon)) + 1

    para_lmin[para_lmin == -np.inf] = -infinity
    para_lmin[para_lmin == np.inf] = infinity
    para_lmax[para_lmax == np.inf] = infinity
    para_lmax[para_lmax == -np.inf] = -infinity

    para_p[para_p == np.inf] = infinity
    para_p[para_p == -np.inf] = -infinity
    para_w[para_w == np.inf] = infinity
    para_w[para_w == -np.inf] = -infinity

    para_a[para_a == np.inf] = infinity
    para_a[para_a == -np.inf] = -infinity

    # =====================================================
    # convert all the numpy arrays with integers data type
    para_lmin = para_lmin.astype(int)
    para_lmax = para_lmax.astype(int)
    para_p = para_p.astype(int)
    para_w = para_w.astype(int)
    para_a = para_a.astype(int)
    para_delta = para_delta.astype(int)
    para_mach_capacity = para_mach_capacity.astype(int)

    para_lmin = para_lmin[:n_opt_selected, :n_opt_selected]
    para_lmax = para_lmax[:n_opt_selected, :n_opt_selected]

    para_p = para_p[:n_opt_selected, :]
    para_h = para_h[:n_opt_selected, :]
    para_w = para_w[:n_opt_selected, :]

    para_a = para_a[:n_opt_selected, :n_opt_selected, :]

    # machines
    machines = [str(i) for i in range(6)]
    # operations
    operations = [str(i) for i in range(n_opt_selected)]

    operations = operations[:n_opt_selected]
    machines = machines[:n_mach]

    # =====================================================
    if method == "CP":
        # time the running time for the CP model
        start_time = time.time()
        fjss2 = FJSS2(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_cp=1.0e6,
            num_workers=4,
            verbose=True,
        )
        fjss2.build_model_ortools()
        fjss2.solve_ortools()
        # get the running time in seconds
        running_time = time.time() - start_time

        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": 0,
            "makespan": fjss2.var_c_max,
        }
    elif method == "MILP":
        para_a = np.einsum("mij->ijm", para_a)
        # time the running time for the MILP model
        start_time = time.time()
        fjss3 = FJSS3(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_milp=1.0e7,
            num_workers=4,
            verbose=True,
        )
        fjss3.solve_gurobi()
        running_time = time.time() - start_time
        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": fjss3.solver.NumConstraints(),
            "makespan": fjss3.var_c_max,
        }
    else:
        raise ValueError("Invalid method!")

I0000 00:00:1705939166.522458 2851015 environment.cc:405] Found the Gurobi library in '/home/fwmeng/softs/gurobi1003/linux64/lib/libgurobi100.so.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-15
Set parameter FeasibilityTol to value 1e-07
Set parameter IntFeasTol to value 1e-07
Set parameter OptimalityTol to value 1e-07
Set parameter Presolve to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen Threadripper PRO 3995WX 64-Cores, instruction set [SSE2|AVX|AVX2]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Optimize a model with 35976 rows, 17901 columns and 193552 nonzeros
Model fingerprint: 0xcc822627
Variable types: 101 continuous, 17800 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+06]
Presolve removed 22562 rows and 15847 columns
Presolve time: 0.15s
Presolved: 13414 rows, 2054 columns, 47320 nonzeros
Variable types: 73 continuous, 1981 integer (1981 binary)

Root relaxation: objective 

In [6]:
solver = fjss3.solver
model = fjss3._model

In [7]:
var_y = fjss3.var_y
# var_y[var_y == 1]

for y in var_y.flatten():
    if y.solution_value() == 1:
        print(y)

y_0_0
y_1_3
y_2_1
y_3_3
y_4_2
y_5_3
y_6_2
y_7_3
y_8_2
y_9_0
y_10_4
y_11_1
y_12_4
y_13_2
y_14_4
y_15_2
y_16_4
y_17_2
y_18_4
y_19_2
y_20_0
y_21_3
y_22_1
y_23_3
y_24_2
y_25_3
y_26_2
y_27_3
y_28_2
y_29_0
y_30_5
y_31_1
y_32_5
y_33_2
y_34_0
y_35_5
y_36_1
y_37_5
y_38_2
y_39_5
y_40_2
y_41_5
y_42_2
y_43_5
y_44_2
y_45_0
y_46_5
y_47_1
y_48_5
y_49_2


In [8]:
var_x = fjss3.var_x
for x in var_x.flatten():
    if x.solution_value() == 1:
        print(x)

x_0_1
x_0_2
x_0_3
x_0_4
x_0_5
x_0_6
x_0_7
x_0_8
x_0_10
x_0_11
x_0_12
x_0_13
x_0_14
x_0_15
x_0_16
x_0_17
x_0_18
x_0_19
x_0_20
x_0_21
x_0_22
x_0_23
x_0_24
x_0_25
x_0_26
x_0_27
x_0_28
x_0_29
x_0_30
x_0_31
x_0_32
x_0_33
x_0_35
x_0_36
x_0_37
x_0_38
x_0_39
x_0_40
x_0_41
x_0_42
x_0_43
x_0_44
x_0_45
x_0_46
x_0_47
x_0_48
x_0_49
x_1_2
x_1_4
x_1_6
x_1_7
x_1_8
x_1_9
x_1_11
x_1_12
x_1_13
x_1_14
x_1_15
x_1_16
x_1_17
x_1_18
x_1_19
x_1_20
x_1_22
x_1_24
x_1_26
x_1_27
x_1_28
x_1_29
x_1_31
x_1_32
x_1_33
x_1_34
x_1_36
x_1_37
x_1_38
x_1_39
x_1_40
x_1_41
x_1_42
x_1_43
x_1_44
x_1_45
x_1_46
x_1_47
x_1_48
x_1_49
x_2_3
x_2_4
x_2_5
x_2_6
x_2_7
x_2_8
x_2_9
x_2_10
x_2_12
x_2_13
x_2_14
x_2_15
x_2_16
x_2_17
x_2_18
x_2_19
x_2_20
x_2_21
x_2_22
x_2_23
x_2_24
x_2_25
x_2_26
x_2_27
x_2_28
x_2_29
x_2_30
x_2_31
x_2_32
x_2_33
x_2_34
x_2_35
x_2_37
x_2_38
x_2_39
x_2_40
x_2_41
x_2_42
x_2_43
x_2_44
x_2_45
x_2_46
x_2_48
x_2_49
x_3_4
x_3_6
x_3_8
x_3_9
x_3_11
x_3_12
x_3_13
x_3_14
x_3_15
x_3_16
x_3_17
x_3_18
x_3_19
x_3_20
x_3_22
x_3

In [9]:
# start time of operation i in machine m
var_s = fjss3.var_s
[var_s[i].solution_value() for i in range(len(var_s))]

[19.999999999999545,
 73.00000000011096,
 352.00000000011096,
 357.00000000011096,
 450.00000000011096,
 460.00000000011096,
 533.000000000111,
 543.000000000111,
 622.0,
 0.0,
 5.0,
 329.0,
 334.0,
 415.0,
 425.0,
 506.0,
 516.0,
 597.0,
 607.0,
 688.0,
 49.999999999999986,
 83.00000000022385,
 362.00000000022385,
 367.00000000022385,
 440.00000000011096,
 450.00000000011096,
 543.0000000002249,
 553.0000000002249,
 632.0000000001146,
 39.99999999999999,
 276.0000000002292,
 557.0000000002292,
 562.0000000002292,
 657.0000000002292,
 9.999999999999545,
 14.999999999999545,
 203.99999999999955,
 208.99999999999955,
 255.99999999999955,
 265.99999999999955,
 312.99999999999955,
 322.99999999999955,
 383.0000000002242,
 403.0000000002242,
 475.0000000002242,
 29.999999999999993,
 35.0,
 224.0,
 229.0,
 276.0]

In [10]:
var_c = fjss3.var_c
[var_c[i].solution_value() for i in range(len(var_c))]

[24.999999999999545,
 347.00000000011096,
 357.00000000011096,
 445.00000000011096,
 460.00000000011096,
 528.000000000111,
 543.000000000111,
 617.0,
 632.0,
 5.0,
 329.0,
 334.0,
 415.0,
 425.0,
 506.0,
 516.0,
 597.0,
 607.0,
 688.0,
 698.0,
 54.999999999999986,
 357.00000000022385,
 367.00000000022385,
 435.00000000011096,
 450.00000000011096,
 538.0000000002249,
 553.0000000002249,
 627.0000000001146,
 642.0000000001146,
 44.99999999999999,
 557.0000000002292,
 562.0000000002292,
 652.0000000002292,
 667.0000000002292,
 14.999999999999545,
 203.99999999999955,
 208.99999999999955,
 255.99999999999955,
 265.99999999999955,
 312.99999999999955,
 322.99999999999955,
 378.0000000002242,
 393.0000000002242,
 470.0000000002242,
 485.0000000002242,
 34.99999999999999,
 224.0,
 229.0,
 276.0,
 286.0]

In [11]:
from checking_constraints import check_constraints_milp, check_constraints_cp

%load_ext autoreload
%autoreload 2

In [12]:
def get_values(var):
    return var.solution_value()


v_get_values = np.vectorize(get_values)

var_y = v_get_values(fjss3.var_y)
var_s = v_get_values(fjss3.var_s)
var_c = v_get_values(fjss3.var_c)
var_c_max = fjss3.var_c_max
var_u = v_get_values(fjss3.var_u)
var_x = v_get_values(fjss3.var_x)
var_z = v_get_values(fjss3.var_z)

check_constraints_cp(
    var_y=var_y,
    var_s=var_s,
    var_c=var_c,
    var_c_max=var_c_max,
    var_u=var_u,
    operations=operations,
    machines=machines,
    para_p=para_p,
    para_a=para_a,
    para_w=para_w,
    para_h=para_h,
    para_delta=para_delta,
    para_mach_capacity=para_mach_capacity,
    para_lmin=para_lmin,
    para_lmax=para_lmax,
    num_t=None,
)

AttributeError: 'FJSS3' object has no attribute 'var_u'